# IceTop Rigidity Analysis

In [1]:
nbins = 50

## Miscellaneous

In [ ]:
# import libraries
import os, sys, glob, pickle, tables

import numpy as np
import matplotlib.pyplot as plt

from pandas import DataFrame

# pip install simweights <--- run this once to install the package
# then replace <username> with your username and <version> with your Python version
sys.path.append('/home/<username>/.local/lib/python3.<version>/site-packages')
import simweights

In [3]:
# Paths to h5 files
PATHS = ['/data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/p/12360_v1s/h5files/*.h5',
'/data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/He/12630_v1s/h5files/*.h5',
'/data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/O/12631_v1s/h5files/*.h5',
'/data/ana/CosmicRay/IceTop_level3/sim/IC86.2012/SIBYLL2.1/Fe/12362_v1s/h5files/*.h5']

In [4]:
# Define important functions
def weighting(path):
    filelist = glob.glob(path)

    for filename in filelist:
        file_obj = tables.open_file(filename, 'r')
    
        if weighter is None:
            weighter = simweights.IceTopWeighter(file_obj)
        else:
            weighter += simweights.IceTopWeighter(file_obj)
    return weighter

def weighted_quantiles(values, weights, quantiles=0.5):
    i = np.argsort(values)
    c = np.cumsum(weights[i])
    return values[i[np.searchsorted(c, np.array(quantiles) * c[-1])]]

def weighted_percentile(data, weights, percentile):
    """
    Compute the weighted percentile of a dataset.
    
    Parameters:
    - data: array-like, the data values.
    - weights: array-like, same length as data.
    - percentile: float between 0 and 100.
    
    Returns:
    - The weighted percentile value.
    """
    data = np.asarray(data)
    weights = np.asarray(weights)
    
    # Sort data and weights by data
    sorted_indices = np.argsort(data)
    data_sorted = data[sorted_indices]
    weights_sorted = weights[sorted_indices]

    # Compute the cumulative sum of weights
    cumulative_weights = np.cumsum(weights_sorted)
    normalized_weights = cumulative_weights / cumulative_weights[-1]  # normalize to 1

    # Find where the normalized cumulative weight exceeds the desired percentile
    return np.interp(percentile / 100, normalized_weights, data_sorted) 

In [5]:
# Load in the SIBYLL2.1 particle sims
weighter = None

# Checks to see if the pickle exists, otherwise prepares weighter
if os.path.isfile('pickles/energy.pkl'):
    with open('pickles/energy.pkl', 'rb') as file:
        primary_energy = pickle.load(file)
else:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

if os.path.isfile('pickles/hits.pkl'):
    with open('pickles/hits.pkl', 'rb') as file:
        hits = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

if os.path.isfile('pickles/particle_type.pkl'):
    with open('pickles/particle_type.pkl', 'rb') as file:
        particle_type = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

if os.path.isfile('pickles/zenith.pkl'):
    with open('pickles/zenith.pkl', 'rb') as file:
        zenith = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

if os.path.isfile('pickles/reco_pass.pkl'):
    with open('pickles/reco_pass.pkl', 'rb') as file:
        reco_pass = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

if os.path.isfile('pickles/Hweights.pkl'):
    with open('pickles/Hweights.pkl', 'rb') as file:
        Hweights = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)
                
if os.path.isfile('pickles/Gweights.pkl'):
    with open('pickles/Gweights.pkl', 'rb') as file:
        Gweights = pickle.load(file)
elif not weighter == None:
    for path in PATHS:
        if weighter == None:
            weighter = weighting(path)
        else:
            weighter += weighting(path)

In [6]:
# If there are missing pkl files, load them with weighter
if not os.path.isfile('pickles/energy.pkl'):
    primary_energy = weighter.get_column('MCPrimary', 'energy')

if not os.path.isfile('pickles/particle_type.pkl'):
    particle_type = weighter.get_column('MCPrimary', 'type')
    
if not os.path.isfile('pickles/zenith.pkl'):
    zenith = weighter.get_column('MCPrimary', 'zenith')

if not os.path.isfile('pickles/hits.pkl'):
    hits = weighter.get_column('IceTopHLCSeedRTPulses_SnowUnAttenuated_info', 'nstrings')

if not os.path.isfile('pickles/reco_pass.pkl'):
    reco_pass = weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTop_reco_succeeded')
        
if not os.path.isfile('pickles/Hweights.pkl'):
    # (p, He, N, Al, Fe) version of this model.
    fluxH = simweights.GaisserH4a_IT()

    # Get the weights by passing the flux to the weighter
    Hweights = weighter.get_weights(fluxH)

if not os.path.isfile('pickles/Gweights.pkl'):
    fluxG = simweights.GlobalSplineFit_IT()

    Gweights = weighter.get_weights(fluxG)

In [7]:
# Sets up quality cut (zenith < 55º and the reconstruction succeeded)
quality_cut = np.logical_and(zenith < np.radians(55), reco_pass.astype(bool))

In [8]:
# Dictionary set-ups
# Different compositions
CUTS = {
    'proton': particle_type==2.21200000e+03,
    'helium' : particle_type==1.00002004e+09,
    'oxygen': particle_type==1.00008016e+09,
    'iron': particle_type==1.00026056e+09,
    'true': np.tile(True, len(primary_energy))
}
# amu of different comps
A = {
    'proton': 1.0073,
    'helium': 4.0026,
    'oxygen': 16,
    'iron': 55.845  
}
# Number of protons for different comps
Z = {
    'proton': 1,
    'helium': 2,
    'oxygen': 8,
    'iron': 26  
}
# IceTop Tiers
'''
Tier 1: 2011-14 is 3≤n<5
Tier 2: 2011 is 5≤n<10; 2012-2013 is 5≤n<9; 2014 is 5≤n<8
Tier 3: 2011 is 10≤n<14, from 2012-13 9≤n<16; 2014-15 is 8≤n<15; 
Tier 4: 2011 is 14≤n, 2012-13 16≤n; 2014-15 is 15≤n; 2016-17 is 14≤n; 2018-19 is 13≤n; 2020-21 is 12≤n
'''
TIERS = {
    'Tier 3': np.logical_and(8<=hits, hits<15, zenith<np.radians(55)),
    'Tier 4': np.logical_and(15<=hits, quality_cut)
}
# Graph colors by comp
COLORS = {
    'true' : 'black',
    'proton': 'red',
    'helium': 'orange',
    'oxygen': 'purple',
    'iron': 'blue'
}
# Graph labels by comp
LABEL = {
    'true' : 'Data',
    'proton': 'p',
    'helium': 'He',
    'oxygen': 'O',
    'iron': 'Fe'
}
# Weights
SIMS = {
    'H4a': Hweights,
    'GSF': Gweights
}
# Flux from histogram
FLUX = {
    'true'  : {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'proton': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'helium': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'oxygen': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'iron'  : {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},}
}
# Histogram bins
BINS = {
    'true'  : {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'proton': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'helium': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'oxygen': {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},},
    'iron'  : {'Tier 1': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 2': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 3': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},
               'Tier 4': {'H4a': np.arange(5, 8.5, .1), 'GSF': np.arange(5, 8.5, .1)},}
}
# Linestyle by weights
LINESTYLE = {
    'H4a': None,
    'GSF': '--'
}
# Dictionary for total flux across compositions for each Tier and weight, and for total flux times log(Z) across compositions for each Tier and weight
TOTALS = {
    'numerator'  : {'Tier 3': {'H4a': np.zeros(nbins), 'GSF': np.zeros(nbins)},
                    'Tier 4': {'H4a': np.zeros(nbins), 'GSF': np.zeros(nbins)}},
    'denominator': {'Tier 3': {'H4a': np.zeros(nbins), 'GSF': np.zeros(nbins)},
                    'Tier 4': {'H4a': np.zeros(nbins), 'GSF': np.zeros(nbins)}},
}
# Table for partial compositions, ln(A), and mean log of rigidity, energy, and Z for each Tier and weight
TABLE = {
        'H4a': {'Tier 3': {'proton part': 0, 'helium part': 0, 'oxygen part': 0, 'iron part': 0, 'ln(A)': 0, '<log(R)>': 0, '<log(E)>': 0, '<log(Z)>': 0},
                'Tier 4': {'proton part': 0, 'helium part': 0, 'oxygen part': 0, 'iron part': 0, 'ln(A)': 0, '<log(R)>': 0, '<log(E)>': 0, '<log(Z)>': 0}},
        'GSF': {'Tier 3': {'proton part': 0, 'helium part': 0, 'oxygen part': 0, 'iron part': 0, 'ln(A)': 0, '<log(R)>': 0, '<log(E)>': 0, '<log(Z)>': 0},
                'Tier 4': {'proton part': 0, 'helium part': 0, 'oxygen part': 0, 'iron part': 0, 'ln(A)': 0, '<log(R)>': 0, '<log(E)>': 0, '<log(Z)>': 0}}
}

## $\log_{10}$ weighted counts vs. $\log_{10}$ Energy

In [ ]:
# Prepare the graphs
ncols = len(TIERS)
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, tier_cut, ax in zip(TIERS.keys(), TIERS.values(), axs):
    # Print Tier for table
    print(tier)
    print('--------------------------------------------------')
    
    for weight_name, weights in SIMS.items():
        # Print weights for table
        print(weight_name)

        for comp, comp_cut in CUTS.items():
            # Combine cuts
            combined_cut = np.logical_and(comp_cut, tier_cut)

            # Graph primary energy
            FLUX[comp][tier][weight_name], BINS[comp][tier][weight_name], patches1 = ax.hist(
            np.log10(primary_energy[combined_cut]), weights=weights[combined_cut],
            bins=nbins, linestyle=LINESTYLE[weight_name], log=True, label=f'{LABEL[comp]} ({weight_name})', histtype='step', color=COLORS[comp])

            # Print median energy and its first sigma
            if comp=='proton' or comp=='iron' or comp=='true':
                print(f'    {comp}')
                print(f'        Median: {round(weighted_quantiles(primary_energy[combined_cut]/1000000, weights[combined_cut]), 2)} PeV')
                print(f'        68%: {round(weighted_percentile(primary_energy[combined_cut]/1000000, weights[combined_cut], 16), 2)}-{round(weighted_percentile(primary_energy[combined_cut]/1000000, Hweights[combined_cut], 84), 2)} PeV')
                print()

    # Label axes and add legend and title
    ax.set_xlabel('log10(Energy)')
    ax.set_ylabel('Rates (Hz)')
    ax.set_title(f'Log10 Energy vs Log 10 Weighted Counts for {tier}')
    ax.legend()

## Mean Atomic Charge

In [ ]:
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, ax in zip(TIERS, axs):    
    for weight_name in SIMS:
        for comp in CUTS:
            if not comp=='true':
                # Sum flux at for each comp, Tier and weight times the number of protons for the comp
                TOTALS['numerator'][tier][weight_name] += FLUX[comp][tier][weight_name] * np.log10(Z[comp])

                # Sum flux at for each comp, Tier and weight
                TOTALS['denominator'][tier][weight_name] += FLUX[comp][tier][weight_name]

        # Calculate <Z>
        meanZ = TOTALS['numerator'][tier][weight_name] / TOTALS['denominator'][tier][weight_name]

        # Plot <Z> vs. energy
        ax.plot(BINS['true'][tier][weight_name][:-1], meanZ, label=weight_name, linestyle=LINESTYLE[weight_name])

    ax.set_xlabel('log10(Energy/GeV)')
    ax.set_ylabel('Mean log10(Z)')
    ax.set_title(f'Mean log(Z) vs. Energy for {tier}')

    ax.legend()

## Particle Fraction vs. Energy

In [ ]:
# Particle Fraction vs. Energy
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, ax in zip(TIERS, axs):
    for weight_name in SIMS:
        for comp in CUTS:
            # Plot fractional composition
            if not comp=='true':
                ax.plot(BINS[comp][tier][weight_name][:-1], FLUX[comp][tier][weight_name] / TOTALS['denominator'][tier][weight_name],
                label=f'{LABEL[comp]} ({weight_name})', linestyle=LINESTYLE[weight_name], color=COLORS[comp])

    ax.set_xlabel('log10(Energy/GeV)')
    ax.set_ylabel('Fraction of Particles')
    ax.set_title(f'Particle Fraction vs. Energy for {tier}')

    ax.legend()

## Table of percentage of particles, $\ln(A)$, and mean $\log_{10}$ of Rigidity

In [ ]:
# Table of the overall percentage of particles, ln(A), and mean log of rigidity
for weight_name, weights in SIMS.items():
    # Print weight name for data table
    print(weight_name)
    print()

    for tier, tier_cut in TIERS.items():
        # Start ln_A at zero
        ln_A = 0

        # Combine tier and quality cuts
        combined_cut = tier_cut

        for comp in CUTS:
            if not comp=='true':
                
                # Save the partial compositions to the table
                TABLE[weight_name][tier][f'{comp} part'] = round(np.mean(FLUX[comp][tier][weight_name]) / np.mean(TOTALS['denominator'][tier][weight_name]), 3)

                # Calculate the mean ln(A) with partial compositions
                ln_A += TABLE[weight_name][tier][f'{comp} part'] * np.log(A[comp])

        # Save ln(A) to the table
        TABLE[weight_name][tier]['ln(A)'] = round(ln_A, 3)
        
        # Calculate mean log E (Scaled to TeV), mean log Z, and finally mean log rigidity
        TABLE[weight_name][tier]['<log(E)>'] = round(np.average(np.log10(primary_energy)[combined_cut] - 3, weights=weights[combined_cut]), 3)
        TABLE[weight_name][tier]['<log(Z)>'] = round(np.mean(TOTALS['numerator'][tier][weight_name]) / np.mean(TOTALS['denominator'][tier][weight_name]), 3)
        TABLE[weight_name][tier]['<log(R)>'] = TABLE[weight_name][tier]['<log(E)>'] - TABLE[weight_name][tier]['<log(Z)>']

    # Print out the table for that weight
    print(DataFrame(TABLE[weight_name]))
    print()

In [ ]:
# Rate vs Rigidity
fig, axs = plt.subplots(figsize = (13 * ncols, 8), ncols=ncols)

for tier, tier_cut, ax in zip(TIERS.keys(), TIERS.values(), axs):
    for weight_name, weights in SIMS.items():

        for comp, comp_cut in CUTS.items():
            combined_cut = np.logical_and(comp_cut, tier_cut, quality_cut)
            rigidity = np.log10(primary_energy[combined_cut]) - 3 - np.mean(TOTALS['numerator'][tier][weight_name]) / np.mean(TOTALS['denominator'][tier][weight_name])
            # Graph rate vs. rigidity
            ax.hist(rigidity,weights=weights[combined_cut], bins=nbins, linestyle=LINESTYLE[weight_name], log=True,
                    label=f'{LABEL[comp]} ({weight_name})', histtype='step', color=COLORS[comp])
    
    ax.set_xlabel('Rigidity')
    ax.set_ylabel('Rate (Hz)')
    ax.set_title(f'Rate vs Rigidity for {tier}')
    ax.legend()